In [80]:
import os
import pandas as pd
import cufflinks as cf
from msk_api.load import Loader

cf.set_config_file(offline=False)
loader = Loader("", "", "", os.path.abspath(""))

In [88]:
secid = "YNDX"
sale_date = pd.Timestamp(2019, 12, 20)
show_date = sale_date - pd.DateOffset(days=20)

file_path = loader.get_data_file_path("trade_history", secid)
hist = pd.read_csv(file_path, sep=";", parse_dates=["TRADEDATE"], infer_datetime_format=True, index_col="TRADEDATE").sort_values("TRADEDATE", ascending = True)[["LEGALCLOSEPRICE"]]
hist = hist.loc[hist.index<=sale_date]
sale_price = hist.loc[hist.index == sale_date, "LEGALCLOSEPRICE"].values[0]

file_path = loader.get_data_file_path("dividends_processed", secid)
divs = pd.read_csv(file_path, sep=";", parse_dates=["t2date"], infer_datetime_format=True, index_col="t2date").sort_values("t2date", ascending = True)[["value"]]
divs = divs.loc[divs.index<=sale_date]

res = hist.join(divs)
res["div_sum"] = res["value"].fillna(0).cumsum()
div_sum = res.loc[res.index == sale_date, "div_sum"].values[0]
res["inv_div_sum"] = div_sum - res["div_sum"]
res["year_percent"] = (sale_price - res["LEGALCLOSEPRICE"])*100.0*365.0/(res["LEGALCLOSEPRICE"]*(sale_date - res.index).days)

# res.loc[res.index<=show_date].iplot(y=["year_percent"], colorscale="-greens", fill=True, xTitle="Dates", yTitle="Percents", title="Interest income per annum ({})".format(secid), dimensions=(1400, 800))